In [30]:
from datetime import datetime
from dateutil import tz
from skyfield.api import EarthSatellite
from skyfield.api import Topos, load
import math
import pandas as pd
import plotly.express as px

In [2]:
DAY_FILE_PATH="/share/nas2/data/vault/TLE_daily"

In [3]:
COLUMNS = ["satellite", "day_dt", "day", "tle_dt", "tle_ts", "line1", "line2"]
DTYPES = [str, str, int, str, int, str, str]


### read test:
```python
df = pd.read_csv("/share/nas2/data/vault/TLE_daily/2016/06/30.tab.gz", 
                 names=COLUMNS, sep='\t', compression='gzip',
                 dtype=dict(zip(COLUMNS, DTYPES))
                ) 
df.count()
df.head()
```


In [4]:
def load_day_file(dt):
    df_path = DAY_FILE_PATH + "/" + "%4d/%02d/%02d.tab.gz"%(dt.year, dt.month, dt.day)
    df = pd.read_csv(df_path, 
                 names=COLUMNS, sep='\t', compression='gzip',
                 dtype=dict(zip(COLUMNS, DTYPES))
                ) 
    return df;
#

In [5]:
df = pd.read_csv("/share/nas2/data/vault/TLE_daily/2016/06/30.tab.gz", 
                 names=COLUMNS, sep='\t', compression='gzip',
                 dtype=dict(zip(COLUMNS, DTYPES))
                ) 
df.count()
df.head()


,satellite,day_dt,day,tle_dt,tle_ts,line1,line2
0,1000,2016-06-30,6026,2016-06-27 11:15:21,1467040521,1 01000U 65008B 16179.46899882 .00000021 0...,2 01000 32.1467 333.7511 0009366 165.3909 194...
1,1000,2016-06-30,6026,2016-06-27 11:15:21,1467040521,1 01000U 65008B 16179.46899882 .00000021 0...,2 01000 32.1467 333.7511 0009366 165.3909 194...
2,1000,2016-06-30,6026,2016-06-27 11:15:21,1467040521,1 01000U 65008B 16179.46899882 .00000021 0...,2 01000 32.1467 333.7511 0009366 165.3909 194...
3,10000,2016-06-30,6026,2016-06-30 10:49:53,1467298193,1 10000U 77034A 16182.45131225 -.00000171 0...,2 10000 15.5820 331.7785 0019081 259.0540 28...
4,10002,2016-06-30,6026,2016-06-28 23:10:32,1467169832,1 10002U 77034C 16180.96565494 -.00000126 0...,2 10002 16.1681 333.0471 0296361 5.9346 0...


In [6]:
print(df.iloc[3]["line1"])
print(df.iloc[3]["line2"])


1 10000U 77034A   16182.45131225 -.00000171  00000-0  00000+0 0  1275
2 10000  15.5820 331.7785 0019081 259.0540  28.2803  0.96674507130362


In [7]:
def test_skyfield():
    lat =  45.0
    lon = -176.0
    earth_position = Topos(lat, lon)
    
    ts = load.timescale()
    t = ts.utc(datetime(2016, 6, 30).replace(tzinfo=tz.tzutc()))
    
    line1="1 10000U 77034A   16182.45131225 -.00000171  00000-0  00000+0 0  1275"
    line2="2 10000  15.5820 331.7785 0019081 259.0540  28.2803  0.96674507130362"
    satellite = EarthSatellite(line1, line2, '77034', ts)
    
    difference = satellite - earth_position
    
    topocentric = difference.at(t)
    alt, az, distance = topocentric.altaz()
    
    print(alt.degrees, az.degrees, distance.km)
#
test_skyfield()

51.64629955572665 179.20537344926646 38068.571578729854


In [8]:
# https://rhodesmill.org/skyfield/earth-satellites.html

In [9]:
(datetime(1971, 6, 1) - datetime(1970, 6, 1)).days

365

In [77]:
def satellite_alt_az_days(dt, lat, lon):
    earth_position = Topos(lat, lon)

    ts = load.timescale()
    t = ts.utc(dt.replace(tzinfo=tz.tzutc()))

    def eval_tle(row):
        satellite = EarthSatellite(row["line1"], row["line2"], 'x', ts)
        delta_days = abs((dt - datetime.strptime(row["tle_dt"], "%Y-%m-%d %H:%M:%S")).days)

        difference = satellite - earth_position

        topocentric = difference.at(t)
        alt, az, distance = topocentric.altaz()
        return pd.Series([alt.degrees, az.degrees, delta_days])
    
    df = load_day_file(dt)
    df_alt_az_days = pd.DataFrame(df.apply(eval_tle, axis=1, raw=True))
    df_alt_az_days.columns = ["altitude", "azimuth", "days"]
    df_alt_az_days.reindex()
    return df_alt_az_days
#

df_alt_az_days = satellite_alt_az_days(datetime(2016, 6, 30), 45.0, -176.0)

In [78]:
df_alt_az_days.count()

altitude    15179
azimuth     15179
days        15179
dtype: int64

In [79]:
df_alt_az_days.head()

,altitude,azimuth,days
0,-42.413819,297.656662,2.0
1,-42.413819,297.656662,2.0
2,-42.413819,297.656662,2.0
3,51.646300,179.205373,1.0
4,-32.583116,299.096010,1.0


In [103]:
def hit_quality(df_alt_az_days):
    
    def eval_quality(row):
        ser = None
        days = row[2]
        altitude = row[0]
        if days <= 2.0:
            if altitude > 0.0:
                vals = ["excellent", math.nan]
            else:
                vals = [math.nan, "excellent"]
        elif days <= 14.0:
            if altitude > 0.0:
                vals = ["good", math.nan]
            else:
                vals = [math.nan, "good"]
        elif days <= 56.0:
            if altitude > 0.0:
                vals = ["poor", math.nan]
            else:
                vals = [math.nan, "poor"]
        else:
            vals = [math.nan, "stale"]
        
        return pd.Series(vals)
    
    df_hit_quality = pd.DataFrame(df_alt_az_days.apply(eval_quality, axis=1))
    df_hit_quality.columns = ["hit", "miss"]
    return df_hit_quality
#

In [104]:
df_hit_quality = hit_quality(df_alt_az_days)

In [105]:
df_hit_quality["hit"].value_counts()

excellent    1713
good          165
poor           23
Name: hit, dtype: int64

In [106]:
df_hit_quality["miss"].value_counts()

excellent    11600
good          1001
stale          469
poor           208
Name: miss, dtype: int64

In [116]:
pd.concat([df_hit_quality["hit"].value_counts(), df_hit_quality["miss"].value_counts()], axis=1, sort=False)

,hit,miss
excellent,1713.0,11600
good,165.0,1001
poor,23.0,208
stale,NaN,469


In [128]:
df_alt_az_days_visible = df_alt_az_days[df_alt_az_days["altitude"]>0]

In [129]:
df_alt_az_days_visible.count()

altitude    1901
azimuth     1901
days        1901
dtype: int64

In [130]:
df_alt_az_days_visible.head(5)

,altitude,azimuth,days
3,51.646300,179.205373,1.0
17,48.571486,219.021839,1.0
21,35.295619,324.390831,0.0
22,67.176418,358.140393,1.0
23,67.176418,358.140393,1.0


In [132]:
df_alt_az_days_visible["color"] = 2
df_alt_az_days_visible.loc[(df_alt_az_days_visible["days"] <= 14.0), "color"] = 1
df_alt_az_days_visible.loc[(df_alt_az_days_visible["days"] <= 2.0), "color"] = 0
df_alt_az_days_visible["R"] = 90.0 - df_alt_az_days_visible["altitude"]
#fig = px.scatter_polar(df_alt_az_days_visible, r="R", theta="azimuth", color_discrete_sequence=['black'])
fig = px.scatter_polar(df_alt_az_days_visible, r="R", theta="azimuth", color="color")
fig.show()



/home/kenglish/.conda/envs/vault/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/kenglish/.conda/envs/vault/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

